## Import Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
% pip install wandb

In [ ]:
# Import dependencies

from os import listdir, remove, makedirs
from os.path import join, isdir, isfile
import numpy as np
from skimage import io
from skimage.transform import resize
import random
import tensorflow as tf
from tensorflow.python.ops import variable_scope as vs
import tensorflow.keras.layers as layers
import wandb

In [3]:
wandb.login()

wandb: Currently logged in as: awesomericky (use `wandb login --relogin` to force relogin)


True

In [4]:
# Import python files

from utils import HGN_organize_data, HGN_divide_train_test
from code_HGN_senEM_train import train

## Load Preprocessed Data

In [5]:
relative_data_directory_path = '/content/drive/MyDrive/Speech2Pickup/data_v1.2_single_channel/preprocessed4HGN_speech2pickup.npz'
    
data = np.load(relative_data_directory_path)

img_idx = data['img_idx']
sen_len = data['seq_len']
speech_inputs = data['inputs']
pos_outputs = data['outputs']
sentence = data['sentence']

print('End loading MetaData')

End loading MetaData


## Load data and divide training and test set

In [ ]:
num_data = img_idx.shape[0]

img_path = '/content/drive/MyDrive/Speech2Pickup/train_img'
script_path = '/content/drive/MyDrive/Speech2Pickup/train_heatmap'

total_images, total_heatmaps = HGN_organize_data(img_path, script_path, model_type='speech2pickup');

args = dict()
args['img_idx'] = img_idx
args['speech_inputs'] = speech_inputs
args['pos_outputs'] = pos_outputs
args['num_data'] = num_data

idx_train, idx_test, num_train, num_test, \
train_img_idx, train_speech_inputs, train_pos_outputs, \
test_img_idx, test_speech_inputs, test_pos_outputs = \
HGN_divide_train_test(args, model_type='speech2pickup')

## Start Training

In [ ]:
batch_size = 8
max_epoch = 300
n_mels = 40
time_steps = 303
num_hg_Depth = 4
dim_hg_feat = 256
learning_rate = 0.00001
dropout_rate = 0.0
train_model_number = 1
input_channel_type = 'single'
run_date = '2020-12-07'
encoder_args = {'num_stacks': 3, 'num_channels':[n_mels for i in range(6)], 'kernel_size':3,
                'dropout_rate': dropout_rate, 'activation': 'leaky-relu', 'return_type': 'end'}
input_shapes = (None, n_mels, time_steps)
seed = 1
training_state = True
restore_flag = 1
restore_epoch = 68
# raise ValueError('Check restore_flag and restore_epoch again!! If confident # this code')
"""
1) restore_flag == 0
- Restore encoder model weights only
- restore_epoch == 0

2) restore_flag == 1
- Restore total model weights
- restore_epoch != 0
"""
if restore_flag == 0 and restore_epoch != 0:
    raise ValueError('Check restore_flag and restore_epoch')
elif restore_flag == 1 and restore_epoch == 0:
    raise ValueError('Check restore_flag and restore_epoch')

wandb_config = {'batch_size': batch_size,
                'max_epoch': max_epoch,
                'n_mels': n_mels,
                'num_hg_Depth': num_hg_Depth,
                'dim_hg_feat': dim_hg_feat,
                'learning_rate': learning_rate,
                'dropout_rate': dropout_rate,
                'train_model_number': train_model_number,
                'input_channel_type': input_channel_type,
                'seed': seed,
                'training_state': training_state,
                'num_train': num_train,
                'num_test': num_test,
                'date': run_date}

if restore_flag == 0:
    id = wandb.util.generate_id()
    print('Wandb id: {}'.format(id))
    with open('/content/drive/MyDrive/Speech2Pickup/HGN_senEM_model/wandbID.txt', 'w') as f:
        f.write(str(id))
elif restore_flag == 1:
    id = '2zb9s0k4'
else:
    raise ValueError('Unsupported restore_flag')

wandb_run = wandb.init(project='Speech2Pickup', name='HGN_senEM', id=id, resume="allow", config=wandb_config)

train(img_resize = 256, # must be fixed
      heatmap_resize = 64, # must be fixed
      num_hg_Depth = num_hg_Depth, # good be fixed
      dim_hg_feat = dim_hg_feat, # good be fixed
      n_mels = n_mels,
      time_steps = time_steps,
      encoder_model_path = '/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/SOTA/encoder_model/sentenceEM_encoder_model',
      encoder_args = encoder_args,
      input_shapes = input_shapes,
      seed = seed,
      training_state = training_state,
      batch_size = batch_size, 
      max_epoch = max_epoch,
      num_train = num_train,
      save_stride = 1,
      learning_rate = learning_rate, # good be fixed
      dropout_rate = dropout_rate,
      restore_flag = restore_flag,
      restore_path = '/content/drive/MyDrive/Speech2Pickup/HGN_senEM_model/model.ckpt',
      restore_epoch = restore_epoch,
      total_images = total_images,
      total_heatmaps = total_heatmaps,
      train_speech_inputs = train_speech_inputs,
      train_img_idx = train_img_idx,
      train_pos_outputs = train_pos_outputs,
      model_save_path = '/content/drive/MyDrive/Speech2Pickup/HGN_senEM_model/model.ckpt')

Now ready to start the session.
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Speech2Pickup/HGN_senEM_model/model.ckpt
batch loss : 6.6901644e-05 -> about 42250.821 second left to finish this epoch
batch loss : 0.00015524059 -> about 1076.403 second left to finish this epoch
batch loss : 0.00027797636 -> about 856.999 second left to finish this epoch
batch loss : 7.2907365e-05 -> about 774.076 second left to finish this epoch
batch loss : 6.7375775e-05 -> about 725.973 second left to finish this epoch
batch loss : 8.7269385e-05 -> about 691.058 second left to finish this epoch
batch loss : 7.5884076e-05 -> about 662.934 second left to finish this epoch
batch loss : 5.7783756e-05 -> about 638.519 second left to finish this epoch
batch loss : 2.7670714e-05 -> about 616.777 second left to finish this epoch
batch loss : 7.572993e-05 -> about 596.744 second left to finish this epoch
batch loss : 7.0792485e-05 -> about 577.844 second left to finish this epoch
batch loss : 

In [ ]:
# Reset tensor graph
# Finish wandb process

tf.reset_default_graph()
wandb.finish()

In [ ]:
# Check variable list (debugging)

variables_names = [v.name for v in tf.trainable_variables()]
variables_names